In [ ]:
!nvidia-smi

In [ ]:
# Стандартные библиотеки
import copy
import gc
import logging
import os
import warnings
import json
from datetime import datetime

from matplotlib import pyplot as plt
import numpy as np
import torch

# Импорты из внешних пакетов (Detectron2)
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import load_coco_json
from detectron2.data import detection_utils as utils
from detectron2.data import build_detection_train_loader
from detectron2.engine import DefaultTrainer
from detectron2.evaluation.evaluator import DatasetEvaluator
from detectron2.utils.visualizer import Visualizer
import detectron2.data.transforms as T

In [ ]:
logger = logging.getLogger('detectron2')
logger.setLevel(logging.CRITICAL)

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")

# Paths to Data

In [ ]:
annotations_train_path = 'data/train/annotations_train.json'
annotations_validation_path = 'data/validation/annotations_validation.json'
images_path = 'data/images'
binary_mask_path = 'data/binary_mask_train.npz'
output_path = 'data/'

In [ ]:
def get_time():
    current_datetime = datetime.now()
    formatted_datetime = current_datetime.strftime("%d_%m_%y_%H_%M")
    return formatted_datetime
    

def clear_cache():
    gc.collect()
    torch.cuda.empty_cache()
    gc.collect()


def check_gpu_availability():
    if torch.cuda.is_available():    
        device = torch.device("cuda")
        print('There are %d GPU(s) available.' % torch.cuda.device_count())

        print('We will use the GPU:', torch.cuda.get_device_name(0))
    else:
        print('No GPU available, using the CPU instead.')
        device = torch.device("cpu")
    return device


def show_images(file_id, dataset, metadata):
    example = dataset[file_id]
    image = utils.read_image(example["file_name"], format="RGB")
    plt.figure(figsize=(3,3),dpi=200)
    visualizer = Visualizer(image[:, :, ::-1], metadata=metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(example)
    plt.imshow(vis.get_image()[:, :,::-1])
    plt.show()


def f1_loss(y_actual, y_predicted):
    true_positive = np.sum(y_actual & y_predicted)
    false_positive = np.sum(~y_actual & y_predicted)
    false_negative = np.sum(y_actual & ~y_predicted)
    
    epsilon = 1e-7
    
    precision = true_positive / (true_positive + false_positive + epsilon)
    recall = true_positive / (true_positive + false_negative + epsilon)
    
    f1 = 2 * precision * recall / (precision + recall + epsilon)
    return f1


class custom_mapper:
    def __init__(self, cfg):
        self.transform_list = [
            T.ResizeShortestEdge(
                [cfg.INPUT.MIN_SIZE_TEST, cfg.INPUT.MIN_SIZE_TEST],
                cfg.INPUT.MAX_SIZE_TEST),
            T.RandomBrightness(0.9, 1.1),
            T.RandomContrast(0.9, 1.1),
            T.RandomSaturation(0.9, 1.1),
            T.RandomLighting(0.9)
        ]
        print(f"[custom_mapper]: {self.transform_list}")

    def __call__(self, dataset_dict):
        dataset_dict = copy.deepcopy(dataset_dict)
        image = utils.read_image(dataset_dict["file_name"], format="BGR")
    
        image, transforms = T.apply_transform_gens(self.transform_list, image)
        dataset_dict["image"] = torch.as_tensor(image.transpose(2, 0, 1).astype("float32"))

        annos = [
            utils.transform_instance_annotations(obj, transforms, image.shape[:2])
            for obj in dataset_dict.pop("annotations")
            if obj.get("iscrowd", 0) == 0
        ]

        instances = utils.annotations_to_instances(annos, image.shape[:2])
        dataset_dict["instances"] = utils.filter_empty_instances(instances)
        return dataset_dict


CHECKPOINTS_RESULTS = []

class F1Evaluator(DatasetEvaluator):
    def __init__(self):
        self.loaded_true = np.load(binary_mask_path)
        self.val_predictions = {}
        self.f1_scores = []
        
    def reset(self):
        self.val_predictions = {}
        self.f1_scores = []

    def process(self, inputs, outputs):
        for input, output in zip(inputs, outputs):
            filename = input["file_name"].split("/")[-1]
            if filename != "41_3.JPG":
                true = self.loaded_true[filename].reshape(-1)

                prediction = output['instances'].pred_masks.cpu().numpy()
                mask = np.add.reduce(prediction)
                mask = (mask > 0).reshape(-1)

                self.f1_scores.append(f1_loss(true, mask))

    def evaluate(self):
        global CHECKPOINTS_RESULTS
        result = np.mean(self.f1_scores)
        CHECKPOINTS_RESULTS.append(result)
        return {"meanF1": result}
    

class AugTrainer(DefaultTrainer):
    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=custom_mapper(cfg))
    
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return F1Evaluator()

In [ ]:
DatasetCatalog.register("my_dataset_train",lambda: load_coco_json(annotations_train_path,
                                                                  image_root = images_path,
                                                                  dataset_name="my_dataset_train",
                                                                  extra_annotation_keys=['bbox_mode']))
DatasetCatalog.register("my_dataset_validation",lambda: load_coco_json(annotations_validation_path,
                                                                  image_root = images_path,
                                                                  dataset_name="my_dataset_validation",
                                                                  extra_annotation_keys=['bbox_mode']))

In [ ]:
dataset_train = DatasetCatalog.get("my_dataset_train")
dataset_validation = DatasetCatalog.get("my_dataset_validation")
print(f'Training dataset size (Images): {len(dataset_train)}')

metadata_train = MetadataCatalog.get("my_dataset_train")
metadata_validation = MetadataCatalog.get("my_dataset_validation")
print(f'Validation dataset size (Images): {len(dataset_validation)}')

In [ ]:
show_images(file_id=0, dataset=dataset_train, metadata=metadata_train)

# Model Configuration

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml")) 
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml")

In [ ]:
# Training and validation datasets
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)

# Parallel data loading
cfg.DATALOADER.NUM_WORKERS = 0

# Image compression
cfg.INPUT.MIN_SIZE_TRAIN = 2160
cfg.INPUT.MAX_SIZE_TRAIN = 3130
cfg.INPUT.MIN_SIZE_TEST = cfg.INPUT.MIN_SIZE_TRAIN
cfg.INPUT.MAX_SIZE_TEST = cfg.INPUT.MAX_SIZE_TRAIN

# Decision threshold
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.1

# Validation frequency
cfg.TEST.EVAL_PERIOD = 1000

# Checkpoint frequency
cfg.SOLVER.CHECKPOINT_PERIOD = cfg.TEST.EVAL_PERIOD

# Blue Green Red (BGR) channel format
cfg.INPUT.FORMAT = 'BGR' 

# Batch size
cfg.SOLVER.IMS_PER_BATCH = 1

# Learning rate
cfg.SOLVER.BASE_LR = 0.01

# Learning rate reduction frequency
cfg.SOLVER.STEPS = (1500,)

# Learning rate reduction factor
cfg.SOLVER.GAMMA = 0.1

# Number of training iterations
cfg.SOLVER.MAX_ITER = 17000

# Number of classes in the dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

# Maximum number of words per page
cfg.TEST.DETECTIONS_PER_IMAGE = 1000

# Model output path
time = get_time()
cfg.OUTPUT_DIR = output_path + '/output' + time

# Create the output directory if it doesn't exist
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# Training the model

In [ ]:
trainer = AugTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
del trainer
clear_cache()

# Save validation results

In [ ]:
results = list(enumerate(CHECKPOINTS_RESULTS, start=1))
with open("CHECKPOINTS_RESULTS.txt", "w") as f:
    f.write(str(results))

# Charts

In [ ]:
# Загрузка данных из JSON-файла
with open(output_path + '/output' + time + '/metrics.json', 'r') as f:
    data = [json.loads(line) for line in f]

# Извлекаем список ключей из первой записи (можно использовать любую другую запись)
all_keys = list(data[0].keys())

# Убираем ключи, которые не подходят для построения графика (например, "data_time" и "eta_seconds")
keys_for_plotting = [key for key in all_keys if key not in ["data_time", "eta_seconds"]]

# Группируем ключи по принадлежности к определенной группе
fast_rcnn_keys = ["fast_rcnn/cls_accuracy", "fast_rcnn/false_negative", "fast_rcnn/fg_cls_accuracy"]
mask_rcnn_keys = ["mask_rcnn/accuracy", "mask_rcnn/false_negative", "mask_rcnn/false_positive"]
loss_keys = ['loss_box_reg', 'loss_cls', 'loss_mask', 'loss_rpn_cls', 'loss_rpn_loc']

# Итерируемся по группам ключей и строим графики
for keys, title in zip([fast_rcnn_keys, mask_rcnn_keys, loss_keys], ["Fast R-CNN Metrics", "Mask R-CNN Metrics", "Loss"]):
    for key in keys:
        values = []

        # Итерируемся по данным и добавляем значения для текущего ключа в список
        for entry in data:
            value = entry.get(key)
            if value is not None:
                values.append(value)

        # Строим график
        plt.plot(range(1, len(values) + 1), values, marker='o', label=key)

    plt.xlabel('Record Number')
    plt.ylabel('Value')
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.show()

**Fast R-CNN и Mask R-CNN**

*Fast R-CNN и Mask R-CNN* - это две эволюционные модификации архитектуры *R-CNN* (Region-based Convolutional Neural Network), предназначенные для решения задачи объектной детекции и сегментации объектов на изображениях. Вот основные отличия между ними:

**Цель задачи:**
- **Fast R-CNN:** Основной целью Fast R-CNN является детекция объектов на изображениях. Он предлагает более эффективный способ извлечения признаков и классификации регионов изображения.
- **Mask R-CNN:** Помимо детекции объектов, Mask R-CNN также решает задачу сегментации объектов. То есть, помимо классификации и определения ограничивающих прямоугольников, он генерирует маски для каждого объекта, указывая пиксели, принадлежащие объекту.

**Архитектура:**
- **Fast R-CNN:** Использует Region Proposal Network (RPN) для предложения областей с потенциальными объектами, а затем применяет сверточные слои и ROI pooling для извлечения признаков и классификации.
- **Mask R-CNN:** Расширяет архитектуру Fast R-CNN, добавляя дополнительную ветвь для генерации масок. Эта ветвь генерирует маску для каждого объекта в ROI (Region of Interest).

**Архитектура и Задачи Fast R-CNN и Mask R-CNN**

*Fast R-CNN:* Использует Region Proposal Network (RPN) для предложения областей с потенциальными объектами, а затем применяет сверточные слои и ROI pooling для извлечения признаков и классификации.

*Mask R-CNN:* Расширяет архитектуру Fast R-CNN, добавляя дополнительную ветвь для генерации масок. Эта ветвь генерирует маску для каждого объекта в ROI (Region of Interest).

**Задачи, решаемые моделями:**
- *Fast R-CNN:* Решает задачу детекции объектов и классификации.
- *Mask R-CNN:* Расширяет функциональность Fast R-CNN, добавляя задачу сегментации объектов (генерация масок).

**Процесс работы с масками:**
- *Fast R-CNN:* Не генерирует маски объектов, а только ограничивающие прямоугольники.
- *Mask R-CNN:* Помимо ограничивающих прямоугольников, генерирует маски для каждого объекта, позволяя точно определить пиксели, принадлежащие объекту.

**Производительность:**

- *Fast R-CNN:* Более быстрый по сравнению с предыдущей версией R-CNN за счет использования ROI pooling, что позволяет эффективнее извлекать признаки из регионов интереса.

- *Mask R-CNN:* Более затратный с точки зрения вычислений из-за генерации масок, что делает его медленнее Fast R-CNN. Однако, он предоставляет дополнительную информацию о форме объектов и их расположении на уровне пикселей.

Выбор между Fast R-CNN и Mask R-CNN зависит от конкретных требований задачи. Если требуется не только детекция объектов, но и точная сегментация, то Mask R-CNN предоставляет необходимую функциональность за счет добавления задачи генерации масок. Однако, следует учитывать увеличение вычислительной сложности и времени обучения при использовании Mask R-CNN.

**Fast R-CNN Metrics:**
1. **`fast_rcnn/cls_accuracy` (Fast R-CNN Classification Accuracy):**
   - Эта метрика измеряет точность классификации объектов моделью Fast R-CNN. Точность классификации показывает, как хорошо модель правильно классифицирует объекты среди всех обнаруженных объектов.

2. **`fast_rcnn/false_negative` (Fast R-CNN False Negatives):**
   - Эта метрика измеряет количество ложных отрицательных предсказаний модели Fast R-CNN. Ложные отрицательные предсказания возникают, когда модель не обнаруживает объекты, которые на самом деле присутствуют в изображении.

3. **`fast_rcnn/fg_cls_accuracy` (Fast R-CNN Foreground Classification Accuracy):**
   - Данная метрика измеряет точность классификации объектов, которые модель Fast R-CNN рассматривает как передний план (foreground). Передний план обычно относится к регионам изображения, содержащим объекты.

**Mask R-CNN Metrics:**
1. **`mask_rcnn/accuracy` (Mask R-CNN Accuracy):**
   - Эта метрика измеряет общую точность модели Mask R-CNN, включая как классификацию объектов, так и точность сегментации масок. Она оценивает, насколько хорошо модель выполняет обе части задачи.

2. **`mask_rcnn/false_negative` (Mask R-CNN False Negatives):**
   - Эта метрика измеряет количество ложных отрицательных предсказаний для сегментации. Такие предсказания возникают, когда модель не правильно выделяет области объектов на изображении.

3. **`mask_rcnn/false_positive` (Mask R-CNN False Positives):**
   - Метрика измеряет количество ложных положительных предсказаний для сегментации. Ложные положительные предсказания возникают, когда модель неверно выделяет области, которые не являются частью объекта.

Эти метрики позволяют оценить качество работы моделей Fast R-CNN и Mask R-CNN с точки зрения классификации объектов и сегментации масок.


**Loss:**
1. **`loss_box_reg` (Loss for bounding box regression):**
   - Этот лосс измеряет ошибку между предсказанными и истинными координатами ограничивающего прямоугольника (bounding box) объекта. Модель предсказывает координаты (x, y, width, height) ограничивающего прямоугольника, и `loss_box_reg` измеряет, насколько эти предсказанные координаты отличаются от фактических.

2. **`loss_cls` (Classification Loss):**
   - Этот лосс отвечает за классификацию объектов в изображении. Модель предсказывает вероятности принадлежности объектов к различным классам, и `loss_cls` измеряет разницу между предсказанными вероятностями и фактическими метками классов.

3. **`loss_mask` (Mask Prediction Loss):**
   - Для сегментации объектов (например, выделение объектов пикселями) используется `loss_mask`. Модель предсказывает маску каждого объекта, и этот лосс измеряет разницу между предсказанной маской и фактической маской.

4. **`loss_rpn_cls` (Region Proposal Network Classification Loss):**
   - Faster R-CNN использует Region Proposal Network (RPN) для предложения областей, где могут находиться объекты. `loss_rpn_cls` измеряет ошибку в классификации предложенных областей (пропозалов) на объекты и не-объекты.

5. **`loss_rpn_loc` (Region Proposal Network Localization Loss):**
   - Этот лосс отвечает за определение координат ограничивающих прямоугольников для предложенных областей (пропозалов), созданных RPN. Он измеряет ошибку в предсказании координат ограничивающих прямоугольников.

